In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

group_stage_table = pd.read_csv(r'CSVs\Group_Stage_Table.csv' , encoding ='latin-1')
world_cup_matches = pd.read_csv(r'CSVs\World_Cup_Matches.csv', encoding = 'latin-1')
world_cup_teams = pd.read_csv(r'CSVs\World_Cup_Teams.csv', encoding='latin-1')
squad_predictions = pd.read_csv(r'CSVs\Predictions\Predictions_Final.csv', encoding ='latin-1')

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Final Simulation Code Chunk

#######################

# Simulation Setup

np.random.seed(22)
num_sims = 100000

## Creating round dataframes with match information

group_stage_matches = world_cup_matches[(world_cup_matches['Stage']=='Group')]
r_16_matches = world_cup_matches[(world_cup_matches['Group']=='R16')]
r_8_matches = world_cup_matches[(world_cup_matches['Group']=='R8')]
r_4_matches = world_cup_matches[(world_cup_matches['Group']=='Semi-Finals')]
final_match = world_cup_matches[(world_cup_matches['Group']=='Finals')]

group_results = []
knockout_sims = []
r_8_sims = []
r_4_sims = []
final_sims = []

# Group Stage Simulation 

## Need to lookup Squad percentages from Squad_Predictions DF

group_stage_matches['S1%']=pd.merge(group_stage_matches, squad_predictions, how='left', left_on=['Squad 1'], right_on=['Squad'])['% of Points Taken']
group_stage_matches['S2%']=pd.merge(group_stage_matches, squad_predictions, how='left', left_on=['Squad 2'], right_on=['Squad'])['% of Points Taken']

## Calculate Probability of S1 winning based on Log 5 Formula

group_stage_matches['S1_Prob'] =((group_stage_matches['S1%'] - (group_stage_matches['S1%']*group_stage_matches['S2%'])) / (group_stage_matches['S1%']))

## Create Group Stage Match Sims

group_match_sims = []

for index,row in group_stage_matches.iterrows():
    result = np.random.binomial(n=1, p = row.S1_Prob, size=num_sims)
    group_match_sims.append(result)


## Add match sim information into group_results 


for index, row in squad_predictions.iterrows():
    team = row.Squad
    group = row.Group
    team_percent = row["% of Points Taken"]
    
    team_rows = group_stage_matches.loc[(group_stage_matches['Squad 1'] == team)| (group_stage_matches['Squad 2']==team),['Squad 1','Squad 2', 'Group']]
    team_rows['is_squad_1'] = team_rows['Squad 1'] == team
    team_sims = [group_match_sims[i] for i in team_rows.index.to_list()]
    
    team_wins = []
    for x in range(len(team_rows)):
        is_squad_1 = team_rows.is_squad_1.values[x]
        game_wins = [y if is_squad_1 else int(not y) for y in team_sims[x]]
        team_wins.append(game_wins)
        
    team_results = np.sum(team_wins, axis=0)
    
    
    group_results.append({'team':team, 'group':group, 'team_percent':team_percent, 'wins':team_results, 'points':team_results*3})

     
    
#R16 Simulation

###############################
    
for z in range(num_sims):
    if (z > 0) and (z % (num_sims / 100) == 0):
        print(f"Simulation {z} Complete.")
        
    elif z == (num_sims - 1):
        print(f"Simulation Complete - The world cup was simulated {num_sims} times")
        
        
    r16_df = [{'team':x['team'], 'group':x['group'], 'team_percent':x['team_percent'], 'wins':x['wins'][z], 'points':x['points'][z]} for x in group_results] 
    r16_df = pd.DataFrame(r16_df)
    r16_df = r16_df.sort_values(by=['group','points', 'team_percent'], ascending=[True,False,False]).reset_index(drop=True)
    r16_df['rank'] = (r16_df.index % 4) + 1
    r16_df['rank'] = r16_df['rank'].astype(str)
    r16_df['seed'] = r16_df['rank'] + r16_df['group']
    
    r_16_results = pd.merge(r_16_matches, r16_df.loc[:,['team','team_percent','seed']], left_on='Squad 1', right_on='seed')
    r_16_results.drop(['Squad 1'], axis=1, inplace=True)
    r_16_results.rename(columns={'team':'Squad 1','team_percent':'S1%', 'seed':'Squad 1 Seed'}, inplace=True)

    r_16_results = pd.merge(r_16_results, r16_df.loc[:,['team','team_percent','seed']], left_on='Squad 2', right_on='seed')
    r_16_results.drop(['Squad 2'], axis=1, inplace=True)
    r_16_results.rename(columns={'team':'Squad 2','team_percent':'S2%', 'seed':'Squad 2 Seed'}, inplace=True)

    r_16_results['S1_Prob'] =((r_16_results['S1%'] - (r_16_results['S1%']*r_16_results['S2%'])) / (r_16_results['S1%'] + r_16_results['S2%'] - (2 * r_16_results['S1%'] * r_16_results['S2%'])))

    r_16_results['S1_wins'] = [np.random.binomial(n=1, p=x, size=1)[0] for x in r_16_results.S1_Prob]
    r_16_results['Simulation'] = z + 1
    knockout_sims.append(r_16_results)

#R8 Simulation

###############################

    r8_df = []
    for index, row in r_16_results.iterrows():
        match = row.Match
        seed = 'M'+str(match)
        if row.S1_wins == 1:
            team = row['Squad 1']
            team_percent = row['S1%']
        else:
            team = row['Squad 2']
            team_percent = row['S2%']

        data = {'seed':seed,'team':team,'team_percent':team_percent}
        r8_df.append(data)

    r8_df = pd.DataFrame(r8_df)
    
    r_8_results = pd.merge(r_8_matches, r8_df.loc[:,['team','team_percent','seed']], left_on='Squad 1', right_on='seed')
    r_8_results.drop(['Squad 1'], axis=1, inplace=True)
    r_8_results.rename(columns={'team':'Squad 1','team_percent':'S1%', 'seed':'Squad 1 Seed'}, inplace=True)

    r_8_results = pd.merge(r_8_results, r8_df.loc[:,['team','team_percent','seed']], left_on='Squad 2', right_on='seed')
    r_8_results.drop(['Squad 2'], axis=1, inplace=True)
    r_8_results.rename(columns={'team':'Squad 2','team_percent':'S2%', 'seed':'Squad 2 Seed'}, inplace=True)

    r_8_results['S1_Prob'] =((r_8_results['S1%'] - (r_8_results['S1%']*r_8_results['S2%'])) / (r_8_results['S1%'] + r_8_results['S2%'] - (2 * r_8_results['S1%'] * r_8_results['S2%'])))

    r_8_results['S1_wins'] = [np.random.binomial(n=1, p=x, size=1)[0] for x in r_8_results.S1_Prob]
    r_8_results['Simulation'] = z + 1
    r_8_sims.append(r_8_results)

    
#Semi-finals Simulation
###############################

    r_4_df = []
    for index, row in r_8_results.iterrows():
        match = row.Match
        seed = 'M'+str(match)
        if row.S1_wins == 1:
            team = row['Squad 1']
            team_percent = row['S1%']
        else:
            team = row['Squad 2']
            team_percent = row['S2%']

        data = {'seed':seed,'team':team,'team_percent':team_percent}
        r_4_df.append(data)

    r_4_df = pd.DataFrame(r_4_df)
    
    r_4_results = pd.merge(r_4_matches, r_4_df.loc[:,['team','team_percent','seed']], left_on='Squad 1', right_on='seed')
    r_4_results.drop(['Squad 1'], axis=1, inplace=True)
    r_4_results.rename(columns={'team':'Squad 1','team_percent':'S1%', 'seed':'Squad 1 Seed'}, inplace=True)

    r_4_results = pd.merge(r_4_results, r_4_df.loc[:,['team','team_percent','seed']], left_on='Squad 2', right_on='seed')
    r_4_results.drop(['Squad 2'], axis=1, inplace=True)
    r_4_results.rename(columns={'team':'Squad 2','team_percent':'S2%', 'seed':'Squad 2 Seed'}, inplace=True)

    r_4_results['S1_Prob'] =((r_4_results['S1%'] - (r_4_results['S1%']*r_4_results['S2%'])) / (r_4_results['S1%'] + r_4_results['S2%'] - (2 * r_4_results['S1%'] * r_4_results['S2%'])))

    r_4_results['S1_wins'] = [np.random.binomial(n=1, p=x, size=1)[0] for x in r_4_results.S1_Prob]
    r_4_results['Simulation'] = z + 1
    r_4_sims.append(r_4_results)

# Finals Simulation
###############################
    
    finals_df = []
    for index, row in r_4_results.iterrows():
        match = row.Match
        seed = 'M'+str(match)
        if row.S1_wins == 1:
            team = row['Squad 1']
            team_percent = row['S1%']
        else:
            team = row['Squad 2']
            team_percent = row['S2%']

        data = {'seed':seed,'team':team,'team_percent':team_percent}
        finals_df.append(data)

    finals_df = pd.DataFrame(finals_df)
    
    final_results = pd.merge(final_match, finals_df.loc[:,['team','team_percent','seed']], left_on='Squad 1', right_on='seed')
    final_results.drop(['Squad 1'], axis=1, inplace=True)
    final_results.rename(columns={'team':'Squad 1','team_percent':'S1%', 'seed':'Squad 1 Seed'}, inplace=True)

    final_results = pd.merge(final_results, finals_df.loc[:,['team','team_percent','seed']], left_on='Squad 2', right_on='seed')
    final_results.drop(['Squad 2'], axis=1, inplace=True)
    final_results.rename(columns={'team':'Squad 2','team_percent':'S2%', 'seed':'Squad 2 Seed'}, inplace=True)

    final_results['S1_Prob'] =((final_results['S1%'] - (final_results['S1%']*final_results['S2%'])) / (final_results['S1%'] + final_results['S2%'] - (2 * final_results['S1%'] * final_results['S2%'])))

    final_results['S1_wins'] = [np.random.binomial(n=1, p=x, size=1)[0] for x in final_results.S1_Prob]
    final_results['Simulation'] = z + 1
    final_sims.append(final_results)

Simulation 1000 Complete.
Simulation 2000 Complete.
Simulation 3000 Complete.
Simulation 4000 Complete.
Simulation 5000 Complete.
Simulation 6000 Complete.
Simulation 7000 Complete.
Simulation 8000 Complete.
Simulation 9000 Complete.
Simulation 10000 Complete.
Simulation 11000 Complete.
Simulation 12000 Complete.
Simulation 13000 Complete.
Simulation 14000 Complete.
Simulation 15000 Complete.
Simulation 16000 Complete.
Simulation 17000 Complete.
Simulation 18000 Complete.
Simulation 19000 Complete.
Simulation 20000 Complete.
Simulation 21000 Complete.
Simulation 22000 Complete.
Simulation 23000 Complete.
Simulation 24000 Complete.
Simulation 25000 Complete.
Simulation 26000 Complete.
Simulation 27000 Complete.
Simulation 28000 Complete.
Simulation 29000 Complete.
Simulation 30000 Complete.
Simulation 31000 Complete.
Simulation 32000 Complete.
Simulation 33000 Complete.
Simulation 34000 Complete.
Simulation 35000 Complete.
Simulation 36000 Complete.
Simulation 37000 Complete.
Simulation

In [ ]:
final_sims[0]

In [14]:
group_match_sims[0]

array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
knockout_sims

In [8]:
type(final_sims)

list

In [58]:
len(world_cup_teams)

32

In [13]:
type(group_results)

list

In [87]:
group_results[0]['wins']

array([1, 1, 1, ..., 2, 1, 2])

In [84]:
group_team_stats

[]

In [85]:
empty_dict = {}

In [ ]:
keys = 

In [59]:
listfor i in range(len(world_cup_teams)):
    print(group_results[i]['team'])


Qatar
Iran
South Korea
Japan
Saudi Arabia
Australia
Ghana
Senegal
Tunisia
Cameroon
Morocco
Canada
Mexico
United States
Costa Rica
Brazil
Argentina
Ecuador
Uruguay
Germany
Denmark
Belgium
France
Croatia
Spain
Serbia
England
Switzerland
Netherlands
Portugal
Poland
Wales


In [19]:
sum(group_results[0]['points'])/ num_sims

3.66831

In [ ]:
test = sum(group_results[0].get('wins')) / num_sims

In [ ]:
test

In [ ]:
for i in range(len(world_cup_teams)):
    

In [ ]:
group_stage_team_wins = []
group_stage_team_wins = 

In [ ]:
type(knockout_sims)

In [ ]:
num_sims = 10000

for z in range(num_sims):
    if (z > 0) and (z % (num_sims / 10) == 0):
        print(f"Simulation {z} Complete.")
        
    elif z == (num_sims - 1):
        print(f"Simulation Complete - The world cup was simulated {num_sims} times")

In [9]:
winners = []

for x in final_sims:
    if x.S1_wins.values[0]==1:
        winners.append(x['Squad 1'].values[0])
    else:
        winners.append(x['Squad 2'].values[0])   

In [10]:
from collections import Counter
Counter(winners).most_common()

[('Netherlands', 14770),
 ('Germany', 11142),
 ('Belgium', 8800),
 ('Denmark', 7788),
 ('Portugal', 7110),
 ('Brazil', 7090),
 ('Morocco', 5483),
 ('Spain', 4876),
 ('England', 4528),
 ('Senegal', 4264),
 ('Mexico', 3230),
 ('Iran', 3147),
 ('Serbia', 2796),
 ('France', 2610),
 ('Cameroon', 1948),
 ('United States', 1728),
 ('Argentina', 1503),
 ('Saudi Arabia', 1494),
 ('South Korea', 1194),
 ('Croatia', 850),
 ('Switzerland', 718),
 ('Canada', 518),
 ('Tunisia', 461),
 ('Ecuador', 327),
 ('Japan', 313),
 ('Australia', 302),
 ('Poland', 270),
 ('Costa Rica', 266),
 ('Qatar', 210),
 ('Uruguay', 101),
 ('Wales', 86),
 ('Ghana', 77)]

In [82]:
group_team_stats = []

#Counter(group_results[0]['wins'])

In [73]:
Iran_wins = Counter(group_results[1]['wins'])

In [74]:
group_team_stats.append(Iran_wins)

In [83]:
group_team_stats

[]

In [117]:
import pickle

In [119]:
with open('group_results.pickle', 'wb') as file:
    pickle.dump(group_results, file)

In [116]:
group_results[0]

{'team': 'Qatar',
 'group': 'A',
 'team_percent': 0.5511940401979817,
 'wins': array([1, 1, 1, ..., 2, 1, 2]),
 'points': array([3, 3, 3, ..., 6, 3, 6])}

In [118]:
type(group_results)

list

In [ ]:
winners

In [81]:
r_4_sims[99999]

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Belgium,0.846234,M57,France,0.746058,M58,0.65196,1,100000
1,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Morocco,0.777304,M59,Argentina,0.712171,M60,0.58518,0,100000


In [24]:
r_4_df = pd.concat(r_4_sims)

In [26]:
r_4_df.head()

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Belgium,0.846234,M57,Mexico,0.724164,M58,0.677030,1,1
1,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Spain,0.804894,M59,Denmark,0.795617,M60,0.514508,0,1
0,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Serbia,0.719247,M57,France,0.746058,M58,0.465811,0,2
1,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Portugal,0.802992,M59,Denmark,0.795617,M60,0.511493,1,2
0,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Japan,0.602125,M57,Netherlands,0.865497,M58,0.190404,1,3


In [28]:
r_4_df.reset_index(inplace = True)

In [32]:
r_4_df.drop(['index'], axis=1, inplace=True)

In [33]:
r_4_df

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
2,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Serbia,0.719247,M57,France,0.746058,M58,0.465811,0,2
3,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Portugal,0.802992,M59,Denmark,0.795617,M60,0.511493,1,2
4,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Japan,0.602125,M57,Netherlands,0.865497,M58,0.190404,1,3
5,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Canada,0.615740,M59,Poland,0.598204,M60,0.518371,1,3
6,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Brazil,0.818765,M57,Netherlands,0.865497,M58,0.412482,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Japan,0.602125,M59,Australia,0.578786,M60,0.524116,1,99998
199996,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Cameroon,0.744420,M57,France,0.746058,M58,0.497842,0,99999
199997,62,Knockout Round,Knockout Round,Semi-Finals,Knockout,Japan,0.602125,M59,Denmark,0.795617,M60,0.279934,1,99999
199998,61,Knockout Round,Knockout Round,Semi-Finals,Knockout,Belgium,0.846234,M57,France,0.746058,M58,0.651960,1,100000


In [34]:
final_sims[0]

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,63,Knockout Round,Knockout Round,Finals,Knockout,Belgium,0.846234,M61,Denmark,0.795617,M62,0.585707,1,1


In [35]:
finals_df = pd.concat(final_sims, ignore_index=True)

In [36]:
finals_df

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,63,Knockout Round,Knockout Round,Finals,Knockout,Belgium,0.846234,M61,Denmark,0.795617,M62,0.585707,1,1
1,63,Knockout Round,Knockout Round,Finals,Knockout,France,0.746058,M61,Portugal,0.802992,M62,0.418874,0,2
2,63,Knockout Round,Knockout Round,Finals,Knockout,Japan,0.602125,M61,Canada,0.615740,M62,0.485710,1,3
3,63,Knockout Round,Knockout Round,Finals,Knockout,Netherlands,0.865497,M61,Costa Rica,0.571931,M62,0.828066,0,4
4,63,Knockout Round,Knockout Round,Finals,Knockout,Cameroon,0.744420,M61,Spain,0.804894,M62,0.413843,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,63,Knockout Round,Knockout Round,Finals,Knockout,Germany,0.826111,M61,Morocco,0.777304,M62,0.576468,1,99996
99996,63,Knockout Round,Knockout Round,Finals,Knockout,Denmark,0.795617,M61,Portugal,0.802992,M62,0.488507,1,99997
99997,63,Knockout Round,Knockout Round,Finals,Knockout,Germany,0.826111,M61,Japan,0.602125,M62,0.758411,1,99998
99998,63,Knockout Round,Knockout Round,Finals,Knockout,France,0.746058,M61,Japan,0.602125,M62,0.660017,0,99999


In [37]:
knockout_sims[0]

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,49,Knockout Round,Knockout Round,R16,Knockout,Netherlands,0.865497,1A,United States,0.685027,2B,0.747391,1,1
1,50,Knockout Round,Knockout Round,R16,Knockout,Mexico,0.724164,1C,Australia,0.578786,2D,0.656429,1,1
2,51,Knockout Round,Knockout Round,R16,Knockout,Denmark,0.795617,1D,Poland,0.598204,2C,0.723348,1,1
3,52,Knockout Round,Knockout Round,R16,Knockout,England,0.792579,1B,Senegal,0.760405,2A,0.546278,0,1
4,53,Knockout Round,Knockout Round,R16,Knockout,Germany,0.826111,1E,Belgium,0.846234,2F,0.463303,0,1
5,54,Knockout Round,Knockout Round,R16,Knockout,Cameroon,0.744420,1G,Ghana,0.474311,2H,0.763491,1,1
6,55,Knockout Round,Knockout Round,R16,Knockout,Canada,0.615740,1F,Spain,0.804894,2E,0.279757,0,1
7,56,Knockout Round,Knockout Round,R16,Knockout,Portugal,0.802992,1H,Switzerland,0.631839,2G,0.703701,1,1


In [38]:
r_16_df = pd.concat(knockout_sims, ignore_index=True)

In [39]:
r_16_df

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,49,Knockout Round,Knockout Round,R16,Knockout,Netherlands,0.865497,1A,United States,0.685027,2B,0.747391,1,1
1,50,Knockout Round,Knockout Round,R16,Knockout,Mexico,0.724164,1C,Australia,0.578786,2D,0.656429,1,1
2,51,Knockout Round,Knockout Round,R16,Knockout,Denmark,0.795617,1D,Poland,0.598204,2C,0.723348,1,1
3,52,Knockout Round,Knockout Round,R16,Knockout,England,0.792579,1B,Senegal,0.760405,2A,0.546278,0,1
4,53,Knockout Round,Knockout Round,R16,Knockout,Germany,0.826111,1E,Belgium,0.846234,2F,0.463303,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,52,Knockout Round,Knockout Round,R16,Knockout,Iran,0.732709,1B,Senegal,0.760405,2A,0.463442,1,100000
799996,53,Knockout Round,Knockout Round,R16,Knockout,Spain,0.804894,1E,Belgium,0.846234,2F,0.428445,0,100000
799997,54,Knockout Round,Knockout Round,R16,Knockout,Brazil,0.818765,1G,Ghana,0.474311,2H,0.833529,1,100000
799998,55,Knockout Round,Knockout Round,R16,Knockout,Morocco,0.777304,1F,Japan,0.602125,2E,0.697558,1,100000


In [40]:
r_8_df = pd.concat(r_8_sims, ignore_index=True)

In [41]:
match_df = [r_16_df, r_8_df, r_4_df, finals_df]

In [43]:
all_knockout_matches = pd.concat(match_df, ignore_index=True)

In [44]:
all_knockout_matches.shape

(1499998, 14)

In [120]:
all_knockout_matches.to_csv('CSVs/Predictions/all_knockout_matches.csv', index=False)

In [97]:
all_knockout_matches['Match Winner'] = np.where(all_knockout_matches['S1_wins'] == 1, all_knockout_matches['Squad 1'], all_knockout_matches['Squad 2'])

In [93]:
test_df = all_knockout_matches.head(5)

In [107]:
US_Matches = all_knockout_matches.loc[(all_knockout_matches['Squad 1'] == 'United States') | (all_knockout_matches['Squad 2'] == 'United States')]

In [115]:
US_Matches

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation,Match Winner
0,49,Knockout Round,Knockout Round,R16,Knockout,Netherlands,0.865497,1A,United States,0.685027,2B,0.747391,1,1,Netherlands
11,52,Knockout Round,Knockout Round,R16,Knockout,United States,0.685027,1B,Ecuador,0.569166,2A,0.622112,1,2,United States
27,52,Knockout Round,Knockout Round,R16,Knockout,United States,0.685027,1B,Senegal,0.760405,2A,0.406626,1,4,United States
35,52,Knockout Round,Knockout Round,R16,Knockout,United States,0.685027,1B,Qatar,0.551194,2A,0.639103,1,5,United States
51,52,Knockout Round,Knockout Round,R16,Knockout,United States,0.685027,1B,Senegal,0.760405,2A,0.406626,0,7,Senegal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499945,63,Knockout Round,Knockout Round,Finals,Knockout,Portugal,0.802992,M61,United States,0.685027,M62,0.652065,0,99948,United States
1499951,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,Belgium,0.846234,M62,0.283251,0,99954,Belgium
1499959,63,Knockout Round,Knockout Round,Finals,Knockout,Morocco,0.777304,M61,United States,0.685027,M62,0.616106,1,99962,Morocco
1499977,63,Knockout Round,Knockout Round,Finals,Knockout,Netherlands,0.865497,M61,United States,0.685027,M62,0.747391,1,99980,Netherlands


In [114]:
US_Matches.loc[(US_Matches['Group'] == 'Finals')].value_counts()

Match  Squad 1 Key     Squad 2 Key     Group   Stage     Squad 1        S1%       Squad 1 Seed  Squad 2        S2%       Squad 2 Seed  S1_Prob   S1_wins  Simulation  Match Winner 
63     Knockout Round  Knockout Round  Finals  Knockout  Argentina      0.712171  M61           United States  0.685027  M62           0.532200  0        11624       United States    1
                                                         United States  0.685027  M61           Denmark        0.795617  M62           0.358438  0        36881       Denmark          1
                                                                                                                                                 1        79241       United States    1
                                                                                                                                                          78283       United States    1
                                                                                

In [94]:
test_df

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation
0,49,Knockout Round,Knockout Round,R16,Knockout,Netherlands,0.865497,1A,United States,0.685027,2B,0.747391,1,1
1,50,Knockout Round,Knockout Round,R16,Knockout,Mexico,0.724164,1C,Australia,0.578786,2D,0.656429,1,1
2,51,Knockout Round,Knockout Round,R16,Knockout,Denmark,0.795617,1D,Poland,0.598204,2C,0.723348,1,1
3,52,Knockout Round,Knockout Round,R16,Knockout,England,0.792579,1B,Senegal,0.760405,2A,0.546278,0,1
4,53,Knockout Round,Knockout Round,R16,Knockout,Germany,0.826111,1E,Belgium,0.846234,2F,0.463303,0,1


In [105]:
US_Matches.loc[(US_Matches['Group'] == 'Finals')]

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation,Match Winner
1400007,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,France,0.746058,M62,0.425379,0,10,France
1400015,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,Poland,0.598204,M62,0.593628,1,18,United States
1400046,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,Belgium,0.846234,M62,0.283251,0,49,Belgium
1400062,63,Knockout Round,Knockout Round,Finals,Knockout,Cameroon,0.744420,M61,United States,0.685027,M62,0.572510,0,65,United States
1400100,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,Argentina,0.712171,M62,0.467800,0,103,Argentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499945,63,Knockout Round,Knockout Round,Finals,Knockout,Portugal,0.802992,M61,United States,0.685027,M62,0.652065,0,99948,United States
1499951,63,Knockout Round,Knockout Round,Finals,Knockout,United States,0.685027,M61,Belgium,0.846234,M62,0.283251,0,99954,Belgium
1499959,63,Knockout Round,Knockout Round,Finals,Knockout,Morocco,0.777304,M61,United States,0.685027,M62,0.616106,1,99962,Morocco
1499977,63,Knockout Round,Knockout Round,Finals,Knockout,Netherlands,0.865497,M61,United States,0.685027,M62,0.747391,1,99980,Netherlands


In [95]:
test_df['Match Winner'] = np.where(test_df['S1_wins'] == 1, test_df['Squad 1'], test_df['Squad 2'])

In [96]:
test_df

,Match,Squad 1 Key,Squad 2 Key,Group,Stage,Squad 1,S1%,Squad 1 Seed,Squad 2,S2%,Squad 2 Seed,S1_Prob,S1_wins,Simulation,Match Winner
0,49,Knockout Round,Knockout Round,R16,Knockout,Netherlands,0.865497,1A,United States,0.685027,2B,0.747391,1,1,Netherlands
1,50,Knockout Round,Knockout Round,R16,Knockout,Mexico,0.724164,1C,Australia,0.578786,2D,0.656429,1,1,Mexico
2,51,Knockout Round,Knockout Round,R16,Knockout,Denmark,0.795617,1D,Poland,0.598204,2C,0.723348,1,1,Denmark
3,52,Knockout Round,Knockout Round,R16,Knockout,England,0.792579,1B,Senegal,0.760405,2A,0.546278,0,1,Senegal
4,53,Knockout Round,Knockout Round,R16,Knockout,Germany,0.826111,1E,Belgium,0.846234,2F,0.463303,0,1,Belgium
